Import source

In [2]:
import sys
sys.path.append("src")

Initialize timer

In [4]:
import time
from time import sleep
start_time = time.time()

# 1. Setup Driver

In [6]:
from driver import setup_driver
driver = setup_driver()

# 2. Login KINEO

In [8]:
from login import login
from config import USER_KEY, PASS_KEY

login(driver,
      url = 'https://aforadores.mitma.es/contadorestraficofomento/Login.aspx',
      username = USER_KEY, 
      password = PASS_KEY)

# 3. go to "Informe por horas" 

In [10]:
driver.get('https://aforadores.mitma.es/contadorestraficofomento/InformePorHorasCalzadaCarrilAforo.aspx')

Initialize flag

In [12]:
flag = True

# 4. AP7 BARCELONA SECTOR 4

## Dropdown 'Demarcación'

In [15]:
from dropdown import select_dropdown_value

select_dropdown_value(driver, 
                      input_id = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_I",
                      dropdown_button_id = "ctl00_ContentPlaceHolderDatos_CbDemarcacion_B-1",
                      value = "AP7 BARCELONA SECTOR 4")

# sleep(1)

## ETD values

In [17]:
dropdown_values_etd = ['AP7-125+650-100270000000', 'AP7-129+500-100280000000', 'AP7-134+050-100290000000']

## Download Excel result

In [19]:
from button import click_button, download_excel
from dates import get_week, select_date
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
import pyautogui
from utils import print_elapsed_time, check_no_data_message

# Get days of the past week
week = get_week()

for value_to_select_etd in dropdown_values_etd:

    # Select ETD value
    select_dropdown_value(driver,
                          input_id = 'ctl00_ContentPlaceHolderDatos_CbEtd_I',
                          dropdown_button_id = 'ctl00_ContentPlaceHolderDatos_CbEtd_B-1',
                          value = value_to_select_etd)

    sleep(1)

    # For every weekday
    for i in range(len(week) - 1):

        # Initial/End date selector
        select_date(driver,
                    date_picker_id = 'LbFechaInicio_I',
                    date_str = f'{week[i]} 00:00:00')
        
        select_date(driver,
                    date_picker_id = 'LbFechaFin_I',
                    date_str = f'{week[i + 1]} 00:00:00')
        
        sleep(1)
    
        # Select Desglose value
        select_dropdown_value(driver,
                              input_id = "ctl00_ContentPlaceHolderDatos_CbDesglose_I",
                              dropdown_button_id = "ctl00_ContentPlaceHolderDatos_CbDesglose_B-1",
                              value = "CARRIL")
    
        sleep(1)
    
        # Click 'Ver' button
        click_button(driver, 
                     button_id = "ctl00_ContentPlaceHolderDatos_BtVerListado_I")
    
        # Wait for "LoadingPanel" to appear
        sleep(2)
    
        # Wait until results table is loaded
        while True:
            try:
                # Check if "LoadingPanel" is visible
                WebDriverWait(driver, 1).until( 
                    EC.visibility_of_element_located((By.ID, "LoadingPanel"))
                )
            except TimeoutException:
                break
    
        sleep(2)
    
        # While message "No hay datos para mostrar" is not shown, Excel will be downloaded
        if not check_no_data_message(driver):

            # only need to do it once
            if flag:
                # 80% zoom adjustment
                pyautogui.keyDown('ctrl') ##### per a què això funcioni, cal tenir el firefox obert (no minimitzat). buscar una alternativa.
                pyautogui.press('-')
                pyautogui.press('-')
                pyautogui.keyUp('ctrl')
    
                flag = False
            
            # Click to download Excel
            download_excel(driver,
                           button_id = "ctl00_ContentPlaceHolderDatos_BtExcel_I") 
            
            sleep(2)  # Wait till download is completed 
            
        else:
            print(f"La ETD {value_to_select_etd} no conté dades a la seva taula. No es baixarà cap document Excel.")
    
        # printar elapsed time
        # print_elapsed_time(start_time)
        # print()

driver.quit()

Could not click the button: Message: (1661, 289) is out of bounds of viewport width (1536) and height (739)
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
MoveTargetOutOfBoundsError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:473:5
assertInViewPort@chrome://remote/content/shared/webdriver/Actions.sys.mjs:2131:11
dispatch@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1026:21
dispatch/pendingEvents<@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1845:14
dispatch@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1844:39
dispatch/chainEvents<@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1770:27
dispatch@chrome://remote/content/shared/webdriver/Actions.sys.mjs:1772:7
performActions@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:516:23
receiveMessage@chrome://remote/content/marionette/actors/MarionetteComma

WebDriverException: Message: Failed to decode response from marionette


# 5. AP7 BARCELONA SECTOR 5